In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
# Instead of importing from 'keras_preprocessing', import from 'tensorflow.keras.preprocessing'
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # data augmentation
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model
# Instead of 'tensorflow.keras.layers import preprocessing', use this for image preprocessing layers:

from tensorflow.keras.applications import MobileNetV2

from pathlib import Path
import os.path
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
!pip install -q kaggle  # Kaggle API'yi kurun
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json  # İzinleri ayarlayın

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [7]:
# Dataseti indirin (URL'yi kullanarak)
!kaggle datasets download -d vencerlanz09/pharmaceutical-drugs-and-vitamins-synthetic-images

# Zip dosyasını çıkarın
!unzip -q pharmaceutical-drugs-and-vitamins-synthetic-images.zip -d dataset

Dataset URL: https://www.kaggle.com/datasets/vencerlanz09/pharmaceutical-drugs-and-vitamins-synthetic-images
License(s): CC-BY-SA-3.0
 96% 228M/238M [00:01<00:00, 114MB/s]
100% 238M/238M [00:02<00:00, 124MB/s]


In [8]:
!ls dataset  # Klasör içeriğini listele

'Drug Vision'   ImageClassesCombinedWithCOCOAnnotations


In [13]:
dataset = "Drug Vision/Data Combined" # verini olduğu dosya yolu
image_dir = Path(dataset)

In [14]:
filepaths = list(image_dir.glob(r'**/*.png'))+list(image_dir.glob(r'**/*.jpg'))#dosya yolu
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1],filepaths))

In [15]:
filepaths = pd.Series(filepaths,name='filepath').astype(str)
labels = pd.Series(labels,name='label')

In [21]:
image_df = pd.concat([filepaths,labels],axis=1)

In [ ]:
# veri görselleştime
random_index = np.random.randint(0,len(image_df),25)
fig,axes = plt.subplots(nrows=5,ncols=5,figsize=(11,11))
for i,ax in enumerate(axes.flat):
    ax.imshow(plt.imread(image_df.filepath[random_index[i]]))
    ax.set_title(image_df.label[random_index[i]])
plt.tight_layout()
plt.show()

In [ ]:
# train test split
train_df,test_df = train_test_split(image_df,train_size=0.8,shuffle=True,random_state=1)

In [ ]:
# data augmentation: veri arttırımı
train_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2
)

In [ ]:
test_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

In [ ]:
# eğitim verilerini dataframe'den akısa alalım
train_images = train_generator.flow_from_dataframe(
    dataframe=image_df,
    x_col='filepath',#görüntü yolu
    y_col='label',# hedef etiket, görüntülerün etiketi
    target_size=(224,224),# görüntülerin hedef boyutu
    color_mode='rgb',# görüntülerin renk modunu belirler
    class_mode='categorical' # çoklu sınıf sınıflandırması
    batch_size=64,
    shuffle=True,# görüntüleri karıştırma
    seed=42,#rastgelelik için tohum
    subset='training' # eğitim seti
)


In [ ]:
val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='filepath',#görüntü yolu
    y_col='label',# hedef etiket, görüntülerün etiketi
    target_size=(224,224),# görüntülerin hedef boyutu
    color_mode='rgb',# görüntülerin renk modunu belirler
    class_mode='categorical' # çoklu sınıf sınıflandırması
    batch_size=64,
    shuffle=True,# görüntüleri karıştırma
    seed=42,#rastgelelik için tohum
    subset='validation' # eğitim seti
)

In [ ]:
test_images = train_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='filepath',#görüntü yolu
    y_col='label',# hedef etiket, görüntülerün etiketi
    target_size=(224,224),# görüntülerin hedef boyutu
    color_mode='rgb',# görüntülerin renk modunu belirler
    class_mode='categorical' # çoklu sınıf sınıflandırması
    batch_size=64,
)

In [ ]:
# resize (görüntüleri boyutlandır) ve rescale (normalizasyon)
resize_and_rescale = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Resizing(224,224),# görüntüleri 224x224 boyutunda yeniden boyutlandorma
    tf.keras.layers.experimental.preprocessing.Rescaling(1.0/255) #normalizasyon
])

In [ ]:
# mobilenet-v2 -> önceden eğitilmiş model
pretrained_model = MobileNetV2(
    input_shape=(224,224,3), # girdilerin yani görüntülerin boyutu
    include_top=False,# mobile net sınıflandırma katmani dahil etme
    weights='imagenet', # hangi veri setiyle eğitildiği
)
pretrained_model.trainable = False # önceden eğitilmiş modelin ağırlıklarını değiştirme

In [ ]:
# callback
checkpoint_patch ="pharmaceutical_drugs_and_vitamins_classification_model_checkpoint"
checkpoint_callback = ModelCheckpoint(checkpoint_patch,
                                      save_weights_only = True # modelin sadece ağırlıklarını kaydet
                                      monitor = "val_accuracy"
                                      save_best_only = True)#sadece en iyi modeli kaydet


In [ ]:
# 5 epoch kadar val_accuracy değerinde iyileşme yoksa eğitimi durdur
early_stopping = EarlyStopping(monitor="val_accuracy",
                               patience=5,
                               restore_best_weights=True)

In [ ]:
# build model while using pretrainde model (mobilenet-v2)
inputs = pretrained_model.input
x = resize_and_rescale(inputs)
x = Dense(256,activation='relu')(pretrained_model.output)
x = Dropout(0.2)(x)
x = Dense(256,activation='relu')(x)
x = Dropout(0.2)(x)
outputs = Dense(10,activation='softmax')(x)

model = Model(inputs=inputs,outputs=outputs)



In [ ]:
# compile
model.compile(
    optimizer=Adam(0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# trainig
history = model.fit(
    train_images,
    steps_per_epoch=len(train_images),
    validation_data=val_images,
    validation_steps=len(val_images),
    epochs=50,
    callbacks=[checkpoint_callback,early_stopping]
)

In [ ]:
# testing
loss,accuracy = model.evaluate(test_images,verbose=1)
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")

In [ ]:
plt.figure()
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'],label='accuracy')
plt.plot(history.history['val_accuracy'],label='val_accuracy')
plt.title("Training and Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.grid(True)
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'],label='loss')
plt.plot(history.history['val_loss'],label='val_loss')
plt.title("Training and Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
labels = train_images.class_indices
labels = dict((v,k) for k,v in labels.items())

In [ ]:
pred = [labels[k] for k in pred]

In [ ]:
random_index = np.random.randint(0,len(test_df)-1,15)
fig,axes = plt.subplots(nrows=5,ncols=3,figsize=(11,11))

In [ ]:
for i,ax in enumerate(axes.flat):
    ax.imshow(plt.imread(image_df.filepath[random_index[i]]))
    if test_df.label.iloc[random_index[i]] == pred[random_index[i]]:
        color = "green"
    else:
        color = "red"
    ax.set_title(f"True: {test_df.label.iloc[random_index[i]]}\nPredicted: {pred[random_index[i]]}",color=color)
plt.tight_layout()
plt.show()

In [ ]:
y_test = list(test_df.label)
print(classification_report(y_test,pred))